# L A U N C H E R   
**understanding how the original code works without roboclaws**<br>
+ Following is a quick code I have put together. It is based on the original code but rearranged as a class instead. I could not spend too much time on the code but, the code in itself is besides the point here. The point is to get an insight in what happens under the hood when we run some actions.<br>The focus of this code is primarily the position function where you will see some encoder values corresponding to motions. Here we move one motor at a time but it is critical to understand that a more complex motion involving several motors is a cascade of the position function (remebering of course that motors should not be working all at once - hence the use of the word cascade. Read about buffer and waiting on buffers in the BasicMicro roboclaw examples to achieve that).

In [18]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 14 19:42:47 2020
@author: Quentin Demory
@license: MIT license
"""
from flask import Flask, render_template, request, jsonify
from roboclaw import Roboclaw # This throws a warning but it works fine
import time
import socket
import logging

logging.basicConfig(filename='launcher.log',level=logging.DEBUG)

# Open serial port
# Linux comport name
rc = Roboclaw("/dev/ttyACM0",115200)
# Windows comport name
# rc = Roboclaw("COM8",115200)
rc.Open()
encoders_ready = 1 # set to 1 so that the position method can be tested

# origo = [90.0, 0, 0, 0] # inactive
# actual_coordonates = origo[:] # inactive
# case_open = 0 # inactive

class motor():

    def __init__(self, address, channel, pulses, length, speed_pulses, speed_manual, ready):
        self.address = address
        self.channel = channel
        self.pulses = pulses
        self.length = length
        self.speed_pulses = speed_pulses
        self.speed_manual = speed_manual
        self.ready = ready
    
    def up(self):
        command = [rc.ForwardM1, rc.ForwardM2]
        try:
            command[self.channel](self.address, self.speed_manual)
        except AttributeError:
            print(f"{command[self.channel]}")
    
    def down(self):
        command = [rc.BackwardM1, rc.BackwardM2]
        try:
            command[self.channel](self.address, self.speed_manual)
        except AttributeError:
            print(f"{command[self.channel]}")

    def position(self):
        command = [rc.SpeedDistanceM1, rc.SpeedDistanceM2, rc.ReadEncM1, rc.ReadEncM2]
        if encoders_ready == 0: #Not execute if the encoders are not ready
            return (''), 403 
        try:
            position = request.form.get('pitch_position', type=int)
        except RuntimeError:
            position = int(input('which position do you want to reach: '))
        if position > self.length or position < 0:
            return ('Out of bounds'), 400
        elif position == 0:
            objective = 0
        else:
            objective = int(self.pulses/(self.length/position))
            print(f'objective = {objective}')
        try:
            actual = command[self.channel+2](self.address)[1]
        except AttributeError:
            actual = int(input("What is your starting position? "))*int(self.pulses/self.length) # Set this to any value between 0 and self.pulses
            print(f'starting point: {actual/(self.pulses/self.length)}')
        increment = objective-actual
        if increment >= 0:
            try:
                command[self.channel](self.address,self.speed_pulses,increment,1) #(address, +-speed, pulses, buffer(0=buffered, 1=Execute immediately))
                command[self.channel](self.address,0,0,0) #To avoid deceleration
            except AttributeError:
                print(f'I go {increment} steps')
                print(f'{self.address}, {self.speed_pulses}, {increment}')
#                logger.info(f'{command[self.channel]}')
        else:
            try:
                command[self.channel](self.address,-self.speed_pulses,-increment,1)
                command[self.channel](self.address,0,0,0) #To avoid deceleration
            except AttributeError:
                print(f'I go {increment} steps')
                print(f'{self.address}, {self.speed_pulses}, {increment}')
#                logger.info(f'{command[self.channel]}')
                
    def stop(self):
        command = [rc.ForwardM1, rc.ForwardM2]
        try:
            command[self.channel](self.address, 0)
        except AttributeError:
            print(f"{command[self.channel]}")
    
pitch = motor(0x80, 0, 355000, 90.0, 7000, 127, 70.0)   # pitch
rotation = motor(0x80, 1, 950000, 180.0, 16000, 15, 10.0)  # rotation
lift = motor(0x81, 0, 19000, 130.0, 420, 127, 130.0)    # lift
launch = motor(0x81, 1, 14800, 111.0, 6*13400, 12, 0.0)   # launch
caseL = motor(0x82, 0, 5000, 5.0, 200, 127, 5.0)  # case left
caseR = motor(0x82, 1, 5000, 5.0, 200, 127, 5.0)  # case right 

pitch.up()
# Master_M1.down()
# Slave_M2.position()
print(pitch.address, lift.address, caseL.address)

# launch has more variables but they are only speed variations and different belt positions
# launch_acceleration=(launch_speed_pulses**2)/13400 #Acceleration during launch (pulses/second2)
# launch_max_speed=10           #Maximum launch speed
# launch_min_speed=1            #Minimum launch speed
# launch_max_acceleration=48    #Maximum launch acceleration
# launch_min_acceleration=1     #Minimum launch acceleration
# launch_standby=8000           #Drone position during stand-by
# launch_mount=17000            #Drone position during mounting
# launch_break=21000            #Belt position during breaking
# launch_bottom=0               #Drone position at the back part of the capsule
# launch_connect=2190           #Belt position for touching the upper part """


<bound method Roboclaw.ForwardM1 of <roboclaw.Roboclaw object at 0x703fec50>>
128 129 130


We see here that the output when running this file is the method the script tried to use to perform ```pitch.up()``` but also ouput the address of the pitch, lift, caseL. Let us do a few more of these before you get the gist of it:

In [3]:
pitch.down()

<bound method Roboclaw.BackwardM1 of <roboclaw.Roboclaw object at 0x703fe430>>


In [4]:
pitch.stop()

<bound method Roboclaw.ForwardM1 of <roboclaw.Roboclaw object at 0x703fe430>>


In [5]:
rotation.up()
rotation.down()
rotation.stop()

<bound method Roboclaw.ForwardM2 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.BackwardM2 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.ForwardM2 of <roboclaw.Roboclaw object at 0x703fe430>>


In [6]:
lift.up() # that would be right
lift.down() # that would be left
lift.stop()
launch.up() # that would be forward
launch.down() # that would be backward
launch.stop()
caseL.up() # that would be open
caseL.down() # that would be close
caseL.stop()
caseR.up() # that would be open
caseR.down() # that would be close
caseR.stop()

<bound method Roboclaw.ForwardM1 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.BackwardM1 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.ForwardM1 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.ForwardM2 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.BackwardM2 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.ForwardM2 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.ForwardM1 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.BackwardM1 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.ForwardM1 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.ForwardM2 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.BackwardM2 of <roboclaw.Roboclaw object at 0x703fe430>>
<bound method Roboclaw.ForwardM2 of <roboclaw.Roboclaw object at 0x703fe430>>


In [7]:
launch.speed_pulses

80400

In [8]:
lift.length

130.0

You get it. Now let us look at what happens when we use the position method. **warning** the last line of input is just a control to see that I am manipulating the right roboclaw at the right address. Then I output the speed_pulses, as a second check and then the increment value that the motor will travel, just because.

In [9]:
pitch.position()

which position do you want to reach: 1
objective = 3944
What is your starting position? 0
starting point: 0.0 degrees
I go 3944 steps
128, 7000, 3944


In [10]:
pitch.position()

which position do you want to reach: 45
objective = 177500
What is your starting position? 0
starting point: 0.0 degrees
I go 177500 steps
128, 7000, 177500


In [11]:
pitch.position()

which position do you want to reach: 35
objective = 138055
What is your starting position? 45
starting point: 44.99492957746479 degrees
I go -39425 steps
128, 7000, -39425


In [12]:
pitch.position()

which position do you want to reach: -4


('Out of bounds', 400)

In [13]:
pitch.position()

which position do you want to reach: 91


('Out of bounds', 400)

**What do we learn from this ?**<br>In the case of the pitch actuator, which is mounted backwards (physically it extends to lower the platform and retracts to set it back up) we learn that the author of the original code set positions relative to the $90^{o}$ angle (platform straight up) as it is the lower encoder value since the actuator is retracted and therefore the starting point. So if you type in that you want to go $20^{o}$ your point of arrival will be $70^{o}$. That is not a very efficient writing. If you wish for the platform to go to $20^{o}$, it should get to $20^{o}$ relative to $0^{o}$, the horizon.

In [16]:
rotation.position()

which position do you want to reach: 10
objective = 52777
What is your starting position? 0
starting point: 0.0 degrees
I go 52777 steps
128, 16000, 52777


In [19]:
rotation.position()

which position do you want to reach: 5
objective = 26388
What is your starting position? 10
starting point: 9.998526315789475
I go -26382 steps
128, 16000, -26382


In [21]:
launch.position()

which position do you want to reach: 100
objective = 13333
What is your starting position? 0
starting point: 0.0
I go 13333 steps
129, 80400, 13333


In [22]:
launch.position()

which position do you want to reach: 50
objective = 6666
What is your starting position? 100
starting point: 99.75
I go -6634 steps
129, 80400, -6634


As for as the code currently on the launcher is concerned, you now understand how the encoder framework works. minimum and maximum encoder values are set in BasicMicroMotionStudio where you retract your motor as much as possible, set that to 0 and then make it go forward to the maximum extention and right the encoder value down as maximum value. Anything in between is just a ratio of that frame. if you have spent a bit of time on BasicMicroMotionStudio or making motors move using my ROBOCLAW tutorial and live_test, it is now clear to you that you can by-pass all the heavy scripting of the position method with ```rc.SpeedAccelDeccelPositionM1()```. This method only requires that the maximum and minimum are set and written to the EEPROM (in BasicMicroMotionStudio) and then you can send the motor up and down at which ever speed you like and to which ever position you like within the motor´s specific encoder frame. What´s more, the roboclaw will actively try to remain at the desired position even if, say, you tried to move it by hand. That encoder frame only works with the above name method. if you bluntly send a forward order on a motor that can spin to higher encoder values, you can easily exceed the encoder maximum and the next time you use the ```rc.SpeedAccelDeccelPositionM1()```, your motor will start by returning within its frame of encoder relative to 0. Just something to look out for.

**AD HOC**: obviously if you wan to play around with the script above, you can vhange some value and run cells and see what happens. Otherwise you can copy paste it to your favorite text editor and see how it behaves. You may also want to actually try it with a motor. for that, you do not need to overwrite anything. Just go to BasicMicroMotionStudio and .tune a test motor you want to use, set the roboclaw's address to 131 (or 0x83) and then add that line to the script:

In [23]:
# test_motor = motor(self, address, channel, pulses, length, speed_pulses, speed_manual, ready)
test_motor = motor(131, 0, 100000, 10.0, 60000, 127, 5.0)

In [26]:
print(test_motor.address) # One can have up to 8 addresses
print(test_motor.channel) # 0 is M1, 1 is M2
print(test_motor.pulses) # Maximum position relative to 0
print(test_motor.length) # What that maximum position is physically in cm or degrees
print(test_motor.speed_pulses) # Maximum motor speed QPPS value
print(test_motor.speed_manual) # Comfortable speed of PWM motion. 0 is 0%, 127 is 100% duty cycle
print(test_motor.ready) # Preset position the motor would travel to is we called the PREPARE function - Not used here.

131
0
100000
10.0
60000
127
5.0


In [29]:
test_motor.up()
test_motor.down()
test_motor.stop()

<bound method Roboclaw.ForwardM1 of <roboclaw.Roboclaw object at 0x703fec50>>
<bound method Roboclaw.BackwardM1 of <roboclaw.Roboclaw object at 0x703fec50>>
<bound method Roboclaw.ForwardM1 of <roboclaw.Roboclaw object at 0x703fec50>>


In [30]:
test_motor.position()

which position do you want to reach: 8
objective = 80000
What is your starting position? 0
starting point: 0.0
I go 80000 steps
131, 60000, 80000


In [31]:
test_motor.position()

which position do you want to reach: 5
objective = 50000
What is your starting position? 8
starting point: 8.0
I go -30000 steps
131, 60000, -30000
